# Source and channel coding


Problème résolu ! Le problème venait du type de `"` utilisé. Pour définir `X2`, il faut bien utiliser `'` et non `"`.

## Programme II
On convertit ici une entrée au format `string` en une entrée au format binaire (soit un tableau de `0` et de `1`).

Entrée: `X2` (string) \
Sortie: `X4` (tableau de `0` et de `1`)

In [38]:
% Programme II
%               ***************
%               *             *
% Message X --> * Source Code * --> X2
%               *             *
%               ***************
% X2 is supposed known

% Warning : The output of LempelZivEnco is a 'string'

% Input : X2
X2 = '01101111010001';

clear X3 X4 Y1 Y2;
X4 = zeros(1, length(X2));

for i = 1:length(X2)
    X4(i) = str2num(X2(i));
end

% X4 is a binary sequence "=" X2
X4
X4(1:10)
X2(1:10)



X4 =

     0     1     1     0     1     1     1     1     0     1     0     0     0     1


ans =

     0     1     1     0     1     1     1     1     0     1


ans =

    '0110111101'




## Channel Code
On calcule ici la distance minimale de Hamming avec la matrice génératrice `G`. Et on vérifie la taille de `X4` pour savoir si on doit ajouter des `0` à la fin de `X4`.

In [39]:
%               ****************
%               *              *
% Message X --> * Channel Code * --> X5
%               *              *
%               ****************

% Let G be a generator matrix of a block code
G = [1 0 1 0 1 0; 0 1 0 1 0 1]; % repetition
u = [0 1; 1 1; 1 0];
distmin = min(sum(rem(u * G, 2), 2))

% Code X4
% Checking the length of the sequence
L4 = length(X4);
rest = rem(L4, size(G, 1));

if rest
    X4(L4 + 1:L4 + size(G, 1) - rest) = zeros(1, size(G, 1) - rest);
end

size(X4)



distmin =

     3


ans =

     1    14




Ici, on sépare `X4` en blocs de taille `size(G,1)` (ici vaut 2). Puis on multiplie chaque bloc de code par G, pour les stocker dans `X5`. Enfin, on calcule le code rate entre `X4` et `X5`.
On constate que le code rate est égal à la distance minimale de Hamming, calculé précédemment. Cela nous permet d'affirmer que le code est correct.

In [40]:

% Verification is possible by vec2mat(mat,padded) of Matlab
mat = vec2mat(X4, size(G, 1));
size(mat)
X4(1:10)
mat(1:5, :)

code = rem(mat * G, 2);
X5 = reshape(code', 1, size(code, 1) * size(code, 2));
Rcb = length(X5) / length(X4) % Code rate



ans =

     7     2


ans =

     0     1     1     0     1     1     1     1     0     1


ans =

     0     1
     1     0
     1     1
     1     1
     0     1


Rcb =

     3




## Decode channel
On calcule l'inverse de la matrice `G` et on vérifie que celle-ci est bien son inverse (en multipliant `G` et `Gri`, on obtient bien la matrice identité). Grâce à l'inverse, on peut décoder le message en multipliant `Y1` par `Gri`, une fois `Y1` séparé en blocs de taille `size(G,2)`.

On calcule `Y2` à partir de `Y1`, le message reçu. On constate que `Y2` est égal à `X4`, ce qui prouve que le décodage s'est bien passé.

In [41]:
%                       ***********
%                       *         *
% Message envoyé X5 --> * Channel * --> Y1
%                       *         *
%                       ***********

% Canal sans bruit ==> Y1 = X5
Y1 = X5;

%                     ******************
%                     *                *
% Message reçu Y1 --> * Decode Channel * --> Y2
%                     *                *
%                     ******************

% Decode Y1
Gri = G' * inv(G * G') % right inverse
G * Gri
Ym = vec2mat(Y1, size(G, 2));
deco = Ym * Gri;

Y2 = reshape(deco', 1, size(deco, 1) * size(deco, 2)); % real vector
Y2 = (Y2 > 0.5); % binary vector after a threshold

BER = sum((Y2 - X4) .^ 2) % Bit Error Rate



Gri =

    0.3333         0
         0    0.3333
    0.3333         0
         0    0.3333
    0.3333         0
         0    0.3333


ans =

     1     0
     0     1


BER =

     0




## Canal II
On introduit des erreurs dans le message `X5` via le canal II. `Y1` contient donc `Nbrerr` erreurs.

In [42]:

%                     ************
%                     *          *
% Sent Message X5 --> * Canal II * --> Y1
%                     *          *
%                     ************

% Noisy channel ==> Y1 = X5 + noise
e = zeros(size(X5));

for i = 1:length(X5)
    e(i) = (mod(i, size(G, 2)) == 1);
end

Nbrerr = sum(e)
Y1 = rem(X5 + e, 2);



Nbrerr =

     7




## Decode canal
On calcule `P` car `G` est dans la forme systématique.

In [48]:
%                         ****************
%                         *              *
% Received Message Y1 --> * Decode Canal * --> Y2
%                         *              *
%                         ****************

% Correction Y1
% Control matrix H
% G is in systematic form
k = size(G, 1);
n = size(G, 2);
P = zeros(k, n - k);

for i = 1:k
    P(i, :) = G(i, k + 1:end);
end

% H=[P^T I] is a n-k x k matrix
Pt = P';
I = eye(n - k);
H = zeros(n - k, k);

for i = 1:n - k
    H(i, 1:size(Pt, 2)) = Pt(i, 1:end);
    H(i, size(Pt, 2) + 1:end) = I(i, 1:end);
end

rem(G * H', 2) % G x H^T = 0


Unable to perform assignment because the size of the left side is 1-by-0 and the size of the right side is 1-by-4.



In [44]:
% error check
mat = vec2mat(Y1, size(G, 2));
s = rem(mat * H', 2); % Syndrome
inder = (sum(s, 2) > 0); % error indicator

% error correction
matc = mat; %mat corrected

for i = 1:size(mat, 1)

    if inder(i) % correction
        a = 0;
        b = 0;

        for j = 1:3 %repetition code 2x6
            a = a + mat(i, 2 * j);
            b = b + mat(i, 2 * j - 1);
        end

        for j = 1:3
            matc(i, 2 * j) = ((a / 3) > 0.5);
            matc(i, 2 * j - 1) = ((b / 3) > 0.5);
        end

    end

end

% Y1 corrected
Y1 = reshape(matc', 1, size(matc, 1) * size(matc, 2));
BER = sum((Y1 - X5) .^ 2) % Bit Error Rate

% Decode Y1
deco = matc * Gri;

Y2 = reshape(deco', 1, size(deco, 1) * size(deco, 2)); % real vector
Y2 = (Y2 > 0.5); % binary vector after a threshold

BER = sum((Y2 - X4) .^ 2) % Bit Error Rate


Incorrect dimensions for matrix multiplication. Check that the number of columns in the first matrix matches the number of rows in the second matrix. To operate on each element of the matrix individually, use TIMES (.*) for elementwise multiplication.



In [45]:
%                              *****************
%                              *               *
% Channel Decoding Y2 = X2 --> * Decode Source * --> Y3 = X
%                              *               *
%                              *****************

% format string
str = num2str(Y2);

for i = 1:length(Y2)
    Y2s(i) = str(3 * (i - 1) + 1);
end

Y3 = LempelZivDeco(Y2s, Dict2); % Y2s doit être un string

% Attention : La sortie de LempelZivDeco est un 'string'
for i = 1:length(Y3)
    Y4(i) = str2num(Y3(i));
end

BERF = sum((X(1:length(Y3)) - Y4) .^ 2) % Bit Error Rate


Unrecognized function or variable 'Dict2'.

